# Heart rate estimation via Template Matching
In this session you will see a practical example of a linear systems modeling approach towards the estimation of the heart rate using a very simple pattern recognition technique called **Template Matching**.

### Modeling assumptions
We will model our acquisition device (e.g., ECG measurement device) as a linear shift invariant system with a given impulse response as illustrated below:

![](ecg_signal_generation_model.png)

### Signal generation

* **Signal Generation -** We are going to create a synthetic EGC signal that we will be using to test our detector algorithm. For that purpose generate a stream of $5$ equally spaced pulses over a time span of $5\ s$ and with a sampling rate of $f_s = 512\ Hz$. You can think of the signal as the convolution of a canonical pulse shape $h(t)$ with a stream of Dirac delta functions: 

   $$y(t) = \sum_{k=1}^K a_k\, h(t-t_k) = h(t) \ast \underbrace{\sum_{k=1}^K a_k\, \delta(t-t_k)}_{x(t)},$$ 

   where $K$ is the number of pulses observed, $a_k$ represent the amplitudes of the different pulses and $t_k$ correspond to the locations of the pulses in time. You can generate the canonical pulse shape using the `ecg_wave()` function.


In [ ]:
# ============================================================================
# import modules
# ============================================================================
# Note that this part of the code needs to be run prior to any other code cell

import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal

# inline plots
%matplotlib inline
sns.set()

# ============================================================================
# functions
# ============================================================================
def ecg_wave(fs):
    """
    This function generates a synthetic ECG template sampled with a sampling frequency fs.

    """
        
    # sample uniformly
    x = np.arange(fs)/fs
    
    # compute signal (superposition of splines)
    ecg = 0.3 * signal.bspline(3*3*x-7.5,2)\
        + 0.15* signal.bspline(3*4*x-2,3)\
        + signal.bspline(3*2*2*x-6,1) - 0.2 * signal.bspline(3*4*x-5,1) - 0.4 * signal.bspline(3*4*x-7,1)
        
    # temporal scaling so that is spans 1/4 seconds
    return ecg[::4]

* **Simulate Noise -** Generate a noisy version of the synthetic ECG signal generated before by adding Gaussian noise with standard deviation $\sigma=0.3$. Plot the noisy observations. Can you identify the locations of the QRS-complex?

### QRS-complex detection via Template Matching
The goal is to identify the locations of the QRS-complexes in the noisy signal using a template matching approach. That will allow us to estimate the heart rate even in the presence of noise. For that purpose do:

* **Template Matching -** Correlate your noisy observation with the template.
* **Noramilize -** Normalize the signal in the range $[0,1]$ for thresholding purposes using the provided function `normalize_range`.
* **Find Peaks -** Define a threshold value in $[0,1]$ and keep only those values of the signal that are above the given threshold. Once you have done that, perform a non-maximum suppression (i.e., keep a value if it is greater than the previous and following values). To implement the thresholding and non-maximum suppression operations you can use `signal.find_peaks` function.

In [ ]:
def normalize_range(x):
    """
    Normalizes the signal such that its minimum value is 0 and its maximum value is 1,
    provided the signal is not identically zero.
    """
    # check that there are non-zero elements
    if np.any(x):
        
        # subtract minimum
        minx = np.min(x)
        z = x - minx
        
        # divide by max value
        maxz = np.max(z)
        return z/maxz
    
    else:
        
        return x


* **RR Intervals -** Plot the original (synthetic) ECG signal as well as the locations of the peaks to verify your method. Keep in mind to compensate for any delay you might have introduced by filtering. From the peak binary signal, estimate the $RR$ interval sequence $r_n$ and its average value as:

$$ \bar R = \frac{1}{N}\sum_{n=0}^N r_n, $$

where $N$ is the number of peaks detected. Provide an estimate of the average heart beat rate in beats per minute $[bpm]$.
